## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

['/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/haekyucho/projects/python/langchain-kr/.venv/lib/python3.11/site-packages']


True

In [8]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("NHN COMMERCE", set_enable=False)

LangSmith 추적을 하지 않습니다.


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.


In [2]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [10]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [11]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [12]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [14]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.


In [15]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [17]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 마치 어린아이가 배우는 것과 비슷합니다.  아이가 "강아지"라는 단어와 강아지 사진을 여러 번 반복해서 보면서 "강아지"라는 단어가 무엇을 의미하는지 배우는 것처럼, AI 모델도 데이터를 통해 배우죠.\n\n핵심은 **데이터**와 **피드백**입니다.\n\n1. **데이터 입력:**  AI 모델은 방대한 양의 데이터(예: 강아지 사진, 고양이 사진, 설명 등)를 입력받습니다. 이 데이터는 모델이 학습할 "교재"와 같습니다.\n\n2. **모델의 예측:** 모델은 입력된 데이터를 바탕으로 스스로 "추측"을 합니다.  처음에는 아무것도 모르기 때문에 무작위로 추측하거나, 매우 부정확한 예측을 할 수 있습니다.  (예: 강아지 사진을 보고 "고양이"라고 잘못 예측할 수 있습니다.)\n\n3. **피드백 (오차 수정):**  모델의 예측이 정답과 얼마나 다른지, 즉 "오차"를 계산합니다.  정답을 알려주는 "정답지"가 필요한 셈입니다.  이 오차를 바탕으로 모델의 내부 매개변수(가중치와 편향 등)를 조정합니다.  오차가 클수록 매개변수를 크게 조정합니다. 이 과정을 **학습**이라고 합니다.\n\n4. **반복:** 2번과 3번 과정을 수없이 반복합니다.  반복 학습을 통해 모델은 오차를 줄이고, 점점 더 정확한 예측을 할 수 있게 됩니다.  마치 아이가 강아지 사진을 많이 볼수록 강아지를 더 잘 구분하게 되는 것과 같습니다.\n\n**간단한 비유:**\n\n활쏘기를 배우는 것을 생각해보세요.\n\n* **데이터:** 과녁과 화살\n* **모델:** 궁수\n* **예측:** 화살이 과녁에 맞는 위치\n* **피드백:** 과녁에 맞은 위치와 중심의 차이 (오차)\n* **학습:** 오차를 줄이기 위해 자세나 힘 조절을 수정하는 것\n\n이 과정을 통해 궁수(모델)는 점점 더 과녁 중심에 화살을 맞출 수 있게 됩니다.  AI 모델도 마찬가지로, 데이터와 피드백을 통해 점점 더 정확한 예측을 할 수 있도록 학습

아래는 스트리밍을 출력하는 예시 입니다.


In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 출력파서(Output Parser)


In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.


In [19]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [20]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
answer = chain.invoke(input)

'인공지능 모델의 학습 원리는 마치 어린아이가 배우는 것과 비슷합니다.  아이가 "강아지"라는 단어와 강아지 사진을 여러 번 보면서 "강아지"라는 단어가 무엇을 의미하는지 배우는 것처럼, AI 모델도 데이터를 통해 배우죠.\n\n핵심은 **데이터**와 **피드백**입니다.\n\n1. **데이터 입력:**  AI 모델은 방대한 양의 데이터를 입력받습니다. 예를 들어, 고양이와 강아지를 구분하는 모델을 학습시키려면 수많은 고양이와 강아지 사진이 필요합니다.  이 데이터에는 사진 자체뿐 아니라 "고양이", "강아지"와 같은 라벨(정답)도 포함됩니다.\n\n2. **모델의 추론:** 모델은 입력된 데이터를 바탕으로 스스로 "고양이"와 "강아지"를 구분하는 방법을 찾아냅니다. 처음에는 무작위로 추측할 수도 있지만, 학습 과정을 거치면서 점점 정확해집니다.  이 과정을 **추론(Inference)**이라고 합니다.\n\n3. **오차 계산:** 모델의 추론 결과와 실제 정답(라벨)을 비교하여 오차(얼마나 틀렸는지)를 계산합니다.  오차가 크다는 것은 모델이 잘못 예측했다는 뜻입니다.\n\n4. **가중치 조정 (학습):**  오차를 줄이기 위해 모델 내부의 **가중치(weight)**와 **편향(bias)**를 조정합니다.  가중치와 편향은 모델이 데이터의 특징을 얼마나 중요하게 고려할지 결정하는 값입니다.  이 조정 과정은 **역전파(Backpropagation)**라는 알고리즘을 통해 이루어집니다.  쉽게 말해, 틀린 부분을 수정하는 과정입니다.\n\n5. **반복:** 1~4번 과정을 수없이 반복합니다.  데이터를 여러 번 반복해서 학습시키면서 오차를 점점 줄여나가고, 모델의 정확도를 높입니다.  이 과정을 **훈련(Training)**이라고 합니다.\n\n결국, AI 모델은 데이터를 통해 패턴을 학습하고, 그 패턴을 이용하여 새로운 데이터에 대해 예측하는 것입니다.  마치 아이가 강아지의 특징(털, 귀, 꼬리 등)을 배우고, 새로운 강아지를 보더라도 강아지라고 인

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI


template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1,
)

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [6]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [7]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
result = chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
print(result)

- **영어 회화:**

**Scenario 1: Simple order**

Waiter: Hi, welcome to [Restaurant Name]. Can I help you?
Me: Yes, please. I'd like the [Dish Name], please.
Waiter:  Okay, and would you like anything to drink?
Me:  Yes, a [Drink Name], please.
Waiter:  Great. I'll be right back with your order.


**Scenario 2:  More detailed order with questions**

Waiter: Hi there, welcome!  What can I get for you today?
Me: Hi.  I'm looking at the [Dish Name].  Does it come with [Side Dish]?
Waiter: Yes, it comes with [Side Dish].  Would you like any substitutions?
Me:  Hmm, could I have [Substitution] instead of [Original Side Dish]?
Waiter: Absolutely. And what about a drink?
Me: I'll have a [Drink Name], please.
Waiter:  Excellent. I'll put your order in.


**Scenario 3:  Asking for recommendations**

Waiter: Hello, welcome to [Restaurant Name].  What are you in the mood for today?
Me:  Hi. I'm not sure, actually. What would you recommend?
Waiter: We have a delicious [Dish Name], it's very popular. Or

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)